In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import threading
import random
import csv

In [ ]:
df = pd.read_csv('good_reads_arabic_books_links.csv')
smaller_dfs = np.array_split(df, 10)
error_link = []
error_number = 0
non_found_link = []

In [ ]:
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:54.0) Gecko/20100101 Firefox/54.0', 
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko', 
    'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15', 
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 OPR/43.0.2442.991', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:65.0) Gecko/20100101 Firefox/65.0', 
    'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0', 
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1', 
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0', 
    'Mozilla/5.0 (Linux; Android 8.0.0; Pixel 2 XL Build/OPD1.170816.004) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Mobile Safari/537.36', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Mobile/15E148 Safari/604.1', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1.2 Safari/605.1.15', 
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.109 Safari/537.36', 
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/601.7.7 (KHTML, like Gecko) Version/9.1.2 Safari/601.7.7'
]

In [ ]:
def get_book_data(driver, link):
    try:
        driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": random.choice(USER_AGENTS)})
        driver.get(link)
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        ButtonElement = driver.find_elements(By.XPATH , '//button[@aria-label="Book details and editions"]')
        try:
            ButtonElement[0].click()
        except Exception:
            try:
                sleep(3)
                ButtonElement[0].click()
            except Exception:
                non_found_link.append(link)
                print(f'-- Not Found link: {link} => Total Not Found {len(non_found_link)}')
                return False

        sleep(2)
        html = driver.page_source
        soup =  BeautifulSoup(html, 'lxml')



        # General Info
        goodreads_id = driver.current_url.rsplit('/', 1)[1]

        try:
            title = soup.find('h1', class_='Text Text__title1').text
        except Exception:
            title = None

        try:
            image = soup.find('div', class_='BookCover').find('img').get('src')
        except Exception:
            image = None

        try:
            goodreads_link = driver.current_url
        except Exception:
            goodreads_link = None

        try:
            publication_info = soup.find('p', {'data-testid': 'publicationInfo'}).text
        except Exception:
            publication_info = None


        # Ratings
        rating_div = soup.find('div', class_='BookPageMetadataSection__ratingStats')
        try:
            number_of_rattings = rating_div.find('span', {'data-testid':'ratingsCount'}).text
        except Exception:
            number_of_rattings = None

        try:
            number_of_reviews = rating_div.find('span', {'data-testid':'reviewsCount'}).text
        except Exception:
            number_of_reviews = None

        try:
            book_rate = rating_div.find('div', class_='RatingStatistics__rating').text
        except Exception:
            book_rate = None

        try:
            number_of_5_stars_rating = soup.find('div', {'class':'RatingsHistogram__labelTotal', 'data-testid': 'labelTotal-5'}).text
        except Exception:
            number_of_5_stars_rating = None

        try:
            number_of_4_stars_rating = soup.find('div', {'class':'RatingsHistogram__labelTotal', 'data-testid': 'labelTotal-4'}).text
        except Exception:
            number_of_4_stars_rating = None

        try:
            number_of_3_stars_rating = soup.find('div', {'class':'RatingsHistogram__labelTotal', 'data-testid': 'labelTotal-3'}).text
        except Exception:
            number_of_3_stars_rating = None

        try:
            number_of_2_stars_rating = soup.find('div', {'class':'RatingsHistogram__labelTotal', 'data-testid': 'labelTotal-2'}).text
        except Exception:
            number_of_2_stars_rating = None

        try:
            number_of_1_stars_rating = soup.find('div', {'class':'RatingsHistogram__labelTotal', 'data-testid': 'labelTotal-1'}).text
        except Exception:
            number_of_1_stars_rating = None


        # Author
        try:
            author_name = soup.find('a', class_='ContributorLink').text
        except Exception:
            author_name = None

        try:
            author_link = soup.find('a', class_='ContributorLink').get('href')
        except Exception:
            author_link = None

        try:
            author_stats = soup.find('div', class_='AuthorPreview').find('span', class_='Text Text__body3 Text__subdued').text
            author_no_of_books = author_stats.split('books')[0]
            author_followers = author_stats.split('books')[1]
        except Exception:
            author_no_of_books = None
            author_followers = None


        # Book Info
        book_info = soup.find_all('div', class_='DescListItem')
        original_title = None
        book_format = None
        published = None
        language = None
        setting = None
        isbn = None
        try:
            for info in book_info:
                if 'Original title' in info.text:
                    original_title = info.find('dd').text
                elif 'Format' in info.text:
                    book_format = info.find('dd').text
                elif 'Published' in info.text:
                    published = info.find('dd').text
                elif 'Language' in info.text:
                    language = info.find('dd').text
                elif 'Setting' in info.text:
                    setting = info.find('dd').text
                elif 'ISBN' in info.text:
                    isbn = info.find('dd').text
        except Exception:
            pass


        # Contributors
        try:
            contributors = soup.find('div', class_='ContributorLinksList').find_all('a')
            all_contributors = []
            for contributor in contributors:
                all_contributors.append(contributor.text)
            all_contributors = ','.join(all_contributors)
        except Exception:
            all_contributors = None


        # Other Info
        try:
            number_of_editions = soup.find('div', class_='MoreEditions').find('a', class_='Button Button--inline Button--small').text
        except Exception:
            number_of_editions = None

        try:
            currently_reading = soup.find('div', {'data-testid': 'currentlyReadingSignal'}).text
        except Exception:
            currently_reading = None

        try:
            to_read = soup.find('div', {'data-testid': 'toReadSignal'}).text
        except Exception:
            to_read = None

        # book genres
        try:
            genres = soup.find_all('span', class_='BookPageMetadataSection__genreButton')
            all_genres = []
            for genre in genres:
                all_genres.append(genre.text)
            all_genres = ','.join(all_genres)
        except Exception:
            all_genres = None

        data = {
            'ISBN':isbn,
            'goodreads_id':goodreads_id,
            'title':title,
            'image':image,
            'goodreads_link':goodreads_link,
            'publication_info':publication_info,
            'number_of_rattings':number_of_rattings,
            'number_of_reviews':number_of_reviews,
            'book_rate':book_rate,
            'number_of_5_stars_rating':number_of_5_stars_rating,
            'number_of_4_stars_rating':number_of_4_stars_rating,
            'number_of_3_stars_rating':number_of_3_stars_rating,
            'number_of_2_stars_rating':number_of_2_stars_rating,
            'number_of_1_stars_rating':number_of_1_stars_rating,
            'author_name':author_name,
            'author_link':author_link,
            'author_no_of_books':author_no_of_books,
            'author_followers':author_followers,
            'original_title':original_title,
            'book_format':book_format,
            'published':published,
            'language':language,
            'setting':setting,
            'all_contributors':all_contributors,
            'number_of_editions':number_of_editions,
            'currently_reading':currently_reading,
            'to_read':to_read,
            'all_genres':all_genres
        }

        return data
    except Exception as e:
        global error_number
        error_number += 1
        print('#' * 50)
        print(f'------ New Error #{error_number} -  link: {link} - Error: {e}')
        print('#' * 50)
        error_data= {link: e}
        error_link.append(error_data)

In [ ]:
def start_scraping(link_all, thread_number, df_index):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--nogpu")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1280,1280")
    chrome_options.add_argument("--enable-javascript")
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    
    # create a new csv per thread to add my data to
    csv_file = open(f'data/goodreads_{df_index}_{thread_number}.csv', 'a+', encoding="utf-16")
    writer = csv.DictWriter(csv_file, fieldnames=['ISBN', 'goodreads_id', 'title', 'image', 'goodreads_link', 'publication_info', 'number_of_rattings', 'number_of_reviews', 'book_rate', 'number_of_5_stars_rating', 'number_of_4_stars_rating', 'number_of_3_stars_rating', 'number_of_2_stars_rating', 'number_of_1_stars_rating', 'author_name', 'author_link', 'author_no_of_books', 'author_followers', 'original_title', 'book_format', 'published', 'language', 'setting', 'all_contributors', 'number_of_editions', 'currently_reading', 'to_read', 'all_genres'], delimiter ='~')
    writer.writeheader()
    
    all_link_num = len(link_all)
    for index, link in enumerate(link_all):
        print(f'DF Index {df_index + 1} - Thread #{thread_number}: Start {index + 1}/{all_link_num}')
        book_data = get_book_data(driver, link)
        if book_data:
            writer.writerow(book_data)
    print('*' * 100)
    print(f'Thread #{thread_number} finshed')
    print('*' * 100)
    driver.close()
        

In [ ]:
for df_index, smaller_df in enumerate(smaller_dfs):
    splited_df = np.array_split(smaller_df, 5) # split my main data frame to 16 data frames (number of threads that I will ran)

    # start runing the threads and gave each one the df that it will scrape
    threads = []
    for i, new_df in enumerate(splited_df):
        thread = threading.Thread(target=start_scraping, kwargs={'link_all': new_df['Link'], 'thread_number': str(i + 1), 'df_index':df_index})
        threads.append(thread)
        thread.start()


    for thread in threads:
        thread.join()

#### Join them in one file

In [ ]:
dfs = []
for i in range(10):
    for j in range(1,6):
        df = pd.read_csv(f'data/goodreads_{i}_{j}.csv', delimiter='~', encoding='utf-16')
        dfs.append(df)

df = pd.concat(dfs)

In [ ]:
df.sort_values('title')

In [ ]:
df.to_csv('goodreads_part_2.csv', encoding='utf-16', index=False)